In [1]:
%%capture
%run merging_data.ipynb

In [ ]:
#concat gpa dfs, filter dfs for only necessary data, change get adm/enrolled/applied a

gpa_all_years = pd.concat(gpa.values(), keys=gpa.keys(), names=['Year'])
gpa_filtered = gpa_all_years[['School', 'Measure Names', 'Measure Values']].reset_index()
gpa_pivot = gpa_filtered.pivot_table(index=['Year', 'School'], columns='Measure Names', values='Measure Values').reset_index()
gpa_annual_avg = gpa_pivot.groupby('Year').mean(numeric_only=True).reset_index()

plt.figure(figsize=(12, 6))

plt.plot(gpa_annual_avg['Year'], gpa_annual_avg['Enrl GPA'], label='Enrolled GPA')
plt.plot(gpa_annual_avg['Year'], gpa_annual_avg['Adm GPA'], label='Admitted GPA')
plt.plot(gpa_annual_avg['Year'], gpa_annual_avg['App GPA'], label='Applied GPA')

plt.xlabel('Year')
plt.ylabel('GPA')
plt.title('GPA Trends Over Years')
plt.legend()
plt.show()



We definitely see an upward trend in the least seven years, but is it a statistically significant increase? Let assess the signifcance by fitting the following regressions.

$$
\text{adm\_gpa} = \beta_0 + \beta_1\text{year}
$$

$$
\text{applied\_gpa} = \beta_0 + \beta_1\text{year}
$$

$$
\text{enrolled\_gpa} = \beta_0 + \beta_1\text{year}
$$

In [ ]:
gpa_annual_avg['Year'] = gpa_annual_avg['Year'].astype(int)

# Prepare the independent variable (years) with an intercept
X = sm.add_constant(gpa_annual_avg['Year'])

# Dependent variables 
y_enrl = gpa_annual_avg['Enrl GPA']
y_adm = gpa_annual_avg['Adm GPA']
y_app = gpa_annual_avg['App GPA']

#Enrolled GPA model
model_enrl = sm.OLS(y_enrl, X).fit()

#Admitted GPA model
model_adm = sm.OLS(y_adm, X).fit()

#Applied GPA model
model_app = sm.OLS(y_app, X).fit()

# model summaries
print("Enrolled GPA Model Summary:")
print(model_enrl.summary())

print("\nAdmitted GPA Model Summary:")
print(model_adm.summary())

print("\nApplied GPA Model Summary:")
print(model_app.summary())

We can see that the gpa of those enrolled and those who applied does 
increase significantly (at the 95% significance level). But the gpa of 
those who were admitted has not increased significantly (at the 95% 
significance level). What do we make of this? Come back to this later.